In [ ]:
from kafka import KafkaProducer
import json

# 连接Kafka
producer = KafkaProducer(
    bootstrap_servers='0.0.0.0:9092',  # Kafka服务器地址
    value_serializer=lambda x: json.dumps(x).encode('utf-8')  # JSON序列化
)

# 生成消息
data = {
    "sno": "95001",
    "name": "John",
    "sex": "M",
    "age": 23
}

# 发送消息
producer.send('json_topic', value=data)
producer.flush()

In [ ]:
from kafka import KafkaProducer
import json

# 连接Kafka
producer = KafkaProducer(
    bootstrap_servers='0.0.0.0:9092',  # Kafka服务器地址
    value_serializer=lambda x: json.dumps(x).encode('utf-8')  # JSON序列化
)

# 生成消息
data = {
    "sno": "95001",
    "name": "John",
    "sex": "M",
    "age": 23
}

# 发送消息
producer.send('json_topic', value=data)
producer.flush()


from kafka import KafkaConsumer
import json
import pymysql.cursors
import time

# ***************************************** Begin ***************************************
# 连接Kafka
consumer = KafkaConsumer(
    'json_topic',  # 主题
    bootstrap_servers='0.0.0.0:9092',  # Kafka服务器地址
    auto_offset_reset='earliest',  # 从最早的消息开始读取
    enable_auto_commit=True,  # 自动提交偏移量
    group_id='my-group',  # 消费者组
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),  # 消息反序列化
    consumer_timeout_ms=1000  # 设置消费者超时时间
)

# 连接数据库
connection = pymysql.connect(
    host='0.0.0.0',
    user='root',
    password='123123',
    database='school',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)

# 遍历数据
for message in consumer:
    # 获取数据
    data = message.value  # 自动解码为字典

    # 连接数据库
    with connection.cursor() as cursor:
        # 插入数据
        sql = "INSERT INTO student (sno, sname, ssex, sage) VALUES (%s, %s, %s, %s)"
        cursor.execute(sql, (data['sno'], data['name'], data['sex'], data['age']))
    
    # 提交事务
    connection.commit()
    
    print(f"Inserted Data: {data}")  # 打印插入的数据

# 关闭数据库连接
connection.close()
# ***************************************** End ***************************************